<a href="https://colab.research.google.com/github/mikaelhonhon/DataExploratory/blob/main/ETL_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
url = "https://id.wikipedia.org/wiki/Daftar_miliarder_Forbes"

In [ ]:
def scrape(url):
  logging.info(f"Scraping website with url: '{url}' ...")
  return pd.read_html(url, header=None)

In [ ]:
dfs = scrape(url)[1]

INFO:root:Scraping website with url: 'https://id.wikipedia.org/wiki/Daftar_miliarder_Forbes' ...


In [ ]:
dfs

,No.,Nama,Kekayaan bersih (USD),Usia,Kebangsaan,Sumber kekayaan
0,NaN,Jeff Bezos,$177 miliar,57,Amerika Serikat,Amazon
1,NaN,Elon Musk,$151 miliar,49,Amerika Serikat,"Tesla, SpaceX"
2,NaN,Bernard Arnault & keluarga,$150 miliar,72,Prancis,LVMH
3,NaN,Bill Gates,$124 miliar,65,Amerika Serikat,Microsoft
4,NaN,Mark Zuckerberg,$97 miliar,36,Amerika Serikat,Meta Platforms
5,NaN,Warren Buffett,$96 miliar,90,Amerika Serikat,Berkshire Hathaway
6,NaN,Larry Ellison,$93 miliar,76,Amerika Serikat,Oracle Corporation
7,NaN,Larry Page,$91.5 miliar,48,Amerika Serikat,Alphabet Inc.
8,NaN,Sergey Brin,$89 miliar,47,Amerika Serikat,Alphabet Inc.
9,NaN,Mukesh Ambani,$84.5 miliar,63,India,Reliance Industries


In [ ]:
import re

In [ ]:
def is_money_miliar(string_money):
  return string_money.lower().endswith('miliar')

In [ ]:
def transform_money_format(string_money):
  half_clean_string = string_money.lower().replace("$","miliar").replace(" ","")

  return re.sub(r"[?\[M\]miliar|\[J\]juta]", "", half_clean_string)

In [ ]:
def transform(df, tahun):
    logging.info("transforming DataFrame ...")
    
    columns_mapping = {
        "No." : "nomor_urut",
        "Nama" : "nama",
        "Kekayaan bersih (USD)": "kekayaan_bersih_usd",
        "Usia" : "usia",
        "Kebangsaan" : "kebangsaan",
        "Sumber kekayaan" : "sumber_kekayaan"
    }

    renamed_df = df.rename(columns = columns_mapping)

    renamed_df ["tahun"] = tahun

    renamed_df["kekayaan_bersih_usd_juta"] = renamed_df["kekayaan_bersih_usd"].apply(
        lambda value: float(transform_money_format(value)) *1000 if is_money_miliar(value) else float(transform_money_format(value))
    )

    return renamed_df[["nomor_urut", "nama", "kekayaan_bersih_usd_juta", "usia", "kebangsaan", "sumber_kekayaan"]]

In [ ]:
df_2021 = transform(dfs, 2021)

INFO:root:transforming DataFrame ...


In [ ]:
df_2021

,nomor_urut,nama,kekayaan_bersih_usd_juta,usia,kebangsaan,sumber_kekayaan
0,NaN,Jeff Bezos,177000.0,57,Amerika Serikat,Amazon
1,NaN,Elon Musk,151000.0,49,Amerika Serikat,"Tesla, SpaceX"
2,NaN,Bernard Arnault & keluarga,150000.0,72,Prancis,LVMH
3,NaN,Bill Gates,124000.0,65,Amerika Serikat,Microsoft
4,NaN,Mark Zuckerberg,97000.0,36,Amerika Serikat,Meta Platforms
5,NaN,Warren Buffett,96000.0,90,Amerika Serikat,Berkshire Hathaway
6,NaN,Larry Ellison,93000.0,76,Amerika Serikat,Oracle Corporation
7,NaN,Larry Page,91500.0,48,Amerika Serikat,Alphabet Inc.
8,NaN,Sergey Brin,89000.0,47,Amerika Serikat,Alphabet Inc.
9,NaN,Mukesh Ambani,84500.0,63,India,Reliance Industries


In [ ]:
pip install psycopg2-binary==2.8.6

     |████████████████████████████████| 3.0 MB 26.2 MB/s 


In [ ]:
from sqlalchemy import create_engine

In [ ]:
DB_NAME = 'postgres'
DB_USER = 'user1'
DB_PASSWORD = 'user1'
DB_HOST = '104.197.148.144'
DB_PORT = '5432'
CONNECTION_STRING = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
TABLE_NAME = 'orang_terkaya_indonesia'

In [ ]:
from sqlalchemy.sql.type_api import TABLEVALUE
def write_to_postgres(df, db_name, table_name, connection_string):
  engine = create_engine(CONNECTION_STRING)
  logging.info(f"Writing dataframe to database: '{db_name}', table:' {table_name}' ...")
  df.to_sql(name = TABLE_NAME, con=engine, if_exists='replace', index=False)

In [ ]:
write_to_postgres(df=df_2021, db_name=DB_NAME, table_name=TABLE_NAME, connection_string=CONNECTION_STRING)

INFO:root:Writing dataframe to database: 'postgres', table:' orang_terkaya_indonesia' ...
